In [1]:
import torch
from brevitas.nn import QuantConv2d, QuantIdentity, QuantMaxPool2d, QuantLinear,QuantDropout,QuantReLU
from brevitas.core.quant import QuantType
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchsummary import summary

### GR LeNet Model

In [12]:
class Gesture_Recognizer(nn.Module):
    def __init__(self):
        super(Gesture_Recognizer,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,
                                padding=0,bias=False)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,
                                padding=0,bias=False)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn3 = nn.Conv2d(in_channels=16,out_channels=120,kernel_size=5,
                                padding=0,bias=False)
        self.relu3 = nn.ReLU()
        
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(1*1*120,180,bias=False)
        self.relu6 = nn.ReLU()
        
        self.fc2 = nn.Linear(180,10,bias=False)
        self.relu7 = nn.ReLU()
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        x = self.relu3(self.cnn3(x))
        
        x = x.view(x.size(0),-1)
        x = self.drop1(x)
        x = self.relu6(self.fc1(x))
        x = self.relu7(self.fc2(x))
        
        return x

In [15]:
GR_LeNet = Gesture_Recognizer()
summary(GR_LeNet,(1,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             150
              ReLU-2            [-1, 6, 28, 28]               0
         MaxPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,400
              ReLU-5           [-1, 16, 10, 10]               0
         MaxPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,000
              ReLU-8            [-1, 120, 1, 1]               0
           Dropout-9                  [-1, 120]               0
           Linear-10                  [-1, 180]          21,600
             ReLU-11                  [-1, 180]               0
           Linear-12                   [-1, 10]           1,800
             ReLU-13                   [-1, 10]               0
Total params: 73,950
Trainable params: 

### ECG 1D Model

In [19]:
class GlobalMaxPool1D(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1D,self).__init__()
    def forward(self,x):
        return torch.max_pool1d(x,kernel_size=x.shape[2])

In [34]:
class ECG_1D_Model(nn.Module):
    def __init__(self):
        super(ECG_1D_Model,self).__init__()
        
        self.cnn1 = nn.Conv1d(in_channels=1,out_channels=18,kernel_size=7,
                                padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2,stride=None)
        
        self.cnn2 = nn.Conv1d(in_channels=18,out_channels=18,kernel_size=7,
                                padding=0)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2,stride=None)    
        
        self.glob_avg_pool = GlobalMaxPool1D() # not MaxPool, should be AveragePool
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(18,6)
        self.relu6 = nn.ReLU()
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        
        x = self.glob_avg_pool(x)
        x = self.drop1(x)
        x = x.view(x.size(0),-1)
        
        x = self.relu6(self.fc1(x))

        
        return x

use nn.AvgPool1d instead of GlobalAveragePool1D

In [2]:
class ECG_1D_Model(nn.Module):
    def __init__(self):
        super(ECG_1D_Model,self).__init__()
        
        self.cnn1 = nn.Conv1d(in_channels=1,out_channels=18,kernel_size=7,
                                padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2,stride=None)
        
        self.cnn2 = nn.Conv1d(in_channels=18,out_channels=18,kernel_size=7,
                                padding=0)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2,stride=None)    
        
        self.glob_avg_pool = nn.AvgPool1d(kernel_size=28)
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(18,6)
        self.relu6 = nn.ReLU()
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        
        x = self.glob_avg_pool(x)
        x = self.drop1(x)
        x = x.view(x.size(0),-1)
        
        x = self.relu6(self.fc1(x))

        
        return x

In [3]:
ECG_1D = ECG_1D_Model()
summary(ECG_1D,(1,130))

D:\Anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 18, 124]             144
              ReLU-2              [-1, 18, 124]               0
         MaxPool1d-3               [-1, 18, 62]               0
            Conv1d-4               [-1, 18, 56]           2,286
              ReLU-5               [-1, 18, 56]               0
         MaxPool1d-6               [-1, 18, 28]               0
         AvgPool1d-7                [-1, 18, 1]               0
           Dropout-8                [-1, 18, 1]               0
            Linear-9                    [-1, 6]             114
             ReLU-10                    [-1, 6]               0
Total params: 2,544
Trainable params: 2,544
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.01
Estimated Total

In [2]:
class ECG_1D_Model(nn.Module):
    def __init__(self):
        super(ECG_1D_Model,self).__init__()
        
        self.cnn1 = nn.Conv1d(in_channels=1,out_channels=18,kernel_size=7,
                                padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2,stride=None)
        
        self.cnn2 = nn.Conv1d(in_channels=18,out_channels=18,kernel_size=7,
                                padding=0)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2,stride=None)    
        
        self.maxpool3 = nn.MaxPool1d(kernel_size=28)
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(18,6)
        self.relu6 = nn.ReLU()
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        
        x = self.maxpool3(x)
        x = self.drop1(x)
        x = x.view(x.size(0),-1)
        
        x = self.relu6(self.fc1(x))

        
        return x

In [4]:
ECG_1D = ECG_1D_Model()
summary(ECG_1D,(1,130))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 18, 124]             144
              ReLU-2              [-1, 18, 124]               0
         MaxPool1d-3               [-1, 18, 62]               0
            Conv1d-4               [-1, 18, 56]           2,286
              ReLU-5               [-1, 18, 56]               0
         MaxPool1d-6               [-1, 18, 28]               0
         MaxPool1d-7                [-1, 18, 1]               0
           Dropout-8                [-1, 18, 1]               0
            Linear-9                    [-1, 6]             114
             ReLU-10                    [-1, 6]               0
Total params: 2,544
Trainable params: 2,544
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.01
Estimated Total

### ECG_AF_1D

In [83]:
class ECG_AF_1D_Model(nn.Module):
    def __init__(self):
        super(ECG_AF_1D_Model,self).__init__()
        
        self.cnn_1 = nn.Conv1d(in_channels=1,out_channels=6,kernel_size=9,stride=3,
                                padding=0,bias=False)
        self.bn_1 = nn.BatchNorm1d(6)
        self.relu_1 = nn.ReLU()
        self.maxpool_1 = nn.MaxPool1d(kernel_size=6,stride=None)
        
        self.cnn_d2 = nn.Conv1d(in_channels=6,out_channels=6,kernel_size=6,
                                padding=0,groups=6,bias=False)
        self.bn_d2 = nn.BatchNorm1d(6)
        self.relu_d2 = nn.ReLU()
        self.cnn_2 = nn.Conv1d(in_channels=6,out_channels=5,kernel_size=1,
                                padding=0,bias=False)
        self.bn_2 = nn.BatchNorm1d(5)
        self.relu_2 = nn.ReLU()
        
        self.cnn_d3 = nn.Conv1d(in_channels=5,out_channels=5,kernel_size=6,
                                padding=0,groups=5,bias=False)
        self.bn_d3 = nn.BatchNorm1d(5)
        self.relu_d3 = nn.ReLU()
        self.cnn_3 = nn.Conv1d(in_channels=5,out_channels=5,kernel_size=1,
                                padding=0,bias=False)
        self.bn_3 = nn.BatchNorm1d(5)
        self.relu_3 = nn.ReLU()
        self.maxpool_3 = nn.MaxPool1d(kernel_size=3,stride=None) 
     
        self.cnn_d4 = nn.Conv1d(in_channels=5,out_channels=5,kernel_size=5,
                                padding=0,groups=5,bias=False)
        self.bn_d4 = nn.BatchNorm1d(5)
        self.relu_d4 = nn.ReLU()
        self.cnn_4 = nn.Conv1d(in_channels=5,out_channels=3,kernel_size=1,
                                padding=0,bias=False)
        self.bn_4 = nn.BatchNorm1d(3)
        self.relu_4 = nn.ReLU()
        self.maxpool_4 = nn.MaxPool1d(kernel_size=6,stride=None) 
        self.maxpool_5 = nn.MaxPool1d(kernel_size=3,stride=None) 
        
        self.drop = nn.Dropout(p=0.5)
        self.dense =nn.Linear(5*3,2,bias=False)
        self.bn_5 = nn.BatchNorm1d(2)
        self.relu_5 = nn.ReLU()
        
    def forward(self,x):
        
        x = self.relu_1(self.bn_1(self.cnn_1(x)))
        x = self.maxpool_1(x)
            
        x = self.relu_d2(self.bn_d2(self.cnn_d2(x)))
        x = self.relu_2(self.bn_2(self.cnn_2(x)))

        x = self.relu_d3(self.bn_d3(self.cnn_d3(x)))
        x = self.relu_3(self.bn_3(self.cnn_3(x)))
        x = self.maxpool_3(x)
        
        x = self.relu_d4(self.bn_d4(self.cnn_d4(x)))
        x = self.relu_4(self.bn_4(self.cnn_4(x)))
        x = self.maxpool_4(x)
        x = self.maxpool_5(x)
            
        x = x.view(x.size(0),-1)
        x = self.drop(x)
        x = self.dense(x)
        x = self.bn_5(x)
        x = self.relu_5(x)

        return x

In [84]:
ECG_AF_1D = ECG_AF_1D_Model()
summary(ECG_AF_1D,(1,5262))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 6, 1752]              54
       BatchNorm1d-2              [-1, 6, 1752]              12
              ReLU-3              [-1, 6, 1752]               0
         MaxPool1d-4               [-1, 6, 292]               0
            Conv1d-5               [-1, 6, 287]              36
       BatchNorm1d-6               [-1, 6, 287]              12
              ReLU-7               [-1, 6, 287]               0
            Conv1d-8               [-1, 5, 287]              30
       BatchNorm1d-9               [-1, 5, 287]              10
             ReLU-10               [-1, 5, 287]               0
           Conv1d-11               [-1, 5, 282]              30
      BatchNorm1d-12               [-1, 5, 282]              10
             ReLU-13               [-1, 5, 282]               0
           Conv1d-14               [-1,

In [3]:
# without depthwise

class ECG_AF_1D_Model(nn.Module):
    def __init__(self):
        super(ECG_AF_1D_Model,self).__init__()
        
        self.cnn_1 = nn.Conv1d(in_channels=1,out_channels=6,kernel_size=9,stride=3,
                                padding=0,bias=False)
        self.bn_1 = nn.BatchNorm1d(6)
        self.relu_1 = nn.ReLU()
        self.maxpool_1 = nn.MaxPool1d(kernel_size=6,stride=None)
        
        self.cnn_d2 = nn.Conv1d(in_channels=6,out_channels=5,kernel_size=5,
                                padding=0,bias=False)
        self.bn_d2 = nn.BatchNorm1d(5)
        self.relu_d2 = nn.ReLU()
        
        self.cnn_d3 = nn.Conv1d(in_channels=5,out_channels=5,kernel_size=6,
                                padding=0,bias=False)
        self.bn_d3 = nn.BatchNorm1d(5)
        self.relu_d3 = nn.ReLU()
        self.maxpool_3 = nn.MaxPool1d(kernel_size=3,stride=None) 
     
        self.cnn_d4 = nn.Conv1d(in_channels=5,out_channels=3,kernel_size=5,
                                padding=0,bias=False)
        self.bn_d4 = nn.BatchNorm1d(3)
        self.relu_d4 = nn.ReLU()
        self.maxpool_4 = nn.MaxPool1d(kernel_size=6,stride=None) 
        self.maxpool_5 = nn.MaxPool1d(kernel_size=3,stride=None) 
        
        self.drop = nn.Dropout(p=0.5)
        self.dense =nn.Linear(5*3,2,bias=False)
        self.bn_5 = nn.BatchNorm1d(2)
        self.relu_5 = nn.ReLU()
        
    def forward(self,x):
        
        x = self.relu_1(self.bn_1(self.cnn_1(x)))
        x = self.maxpool_1(x)
            
        x = self.relu_d2(self.bn_d2(self.cnn_d2(x)))

        x = self.relu_d3(self.bn_d3(self.cnn_d3(x)))
        x = self.maxpool_3(x)
        
        x = self.relu_d4(self.bn_d4(self.cnn_d4(x)))
        x = self.maxpool_4(x)
        x = self.maxpool_5(x)
            
        x = x.view(x.size(0),-1)
        x = self.drop(x)
        x = self.dense(x)
        x = self.bn_5(x)
        x = self.relu_5(x)

        return x

In [4]:
ECG_AF_1D = ECG_AF_1D_Model()
summary(ECG_AF_1D,(1,5262))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 6, 1752]              54
       BatchNorm1d-2              [-1, 6, 1752]              12
              ReLU-3              [-1, 6, 1752]               0
         MaxPool1d-4               [-1, 6, 292]               0
            Conv1d-5               [-1, 5, 288]             150
       BatchNorm1d-6               [-1, 5, 288]              10
              ReLU-7               [-1, 5, 288]               0
            Conv1d-8               [-1, 5, 283]             150
       BatchNorm1d-9               [-1, 5, 283]              10
             ReLU-10               [-1, 5, 283]               0
        MaxPool1d-11                [-1, 5, 94]               0
           Conv1d-12                [-1, 3, 90]              75
      BatchNorm1d-13                [-1, 3, 90]               6
             ReLU-14                [-1

D:\Anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


### ECG_AF_2D

In [98]:
class ECG_AF_2D_Model(nn.Module):
    def __init__(self):
        super(ECG_AF_2D_Model,self).__init__()
        
        self.cnn_1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=2,
                                padding=0,bias=False)
        self.bn_1 = nn.BatchNorm2d(6)
        self.relu_1 = nn.ReLU()
        self.maxpool_1 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn_d2 = nn.Conv2d(in_channels=6,out_channels=6,kernel_size=3,
                                padding=0,groups=6,bias=False)
        self.bn_d2 = nn.BatchNorm2d(6)
        self.relu_d2 = nn.ReLU()
        self.cnn_2 = nn.Conv2d(in_channels=6,out_channels=5,kernel_size=1,
                                padding=0,bias=False)
        self.bn_2 = nn.BatchNorm2d(5)
        self.relu_2 = nn.ReLU()
        
        self.cnn_d3 = nn.Conv2d(in_channels=5,out_channels=5,kernel_size=3,
                                padding=1,groups=5,bias=False)
        self.bn_d3 = nn.BatchNorm2d(5)
        self.relu_d3 = nn.ReLU()
        self.cnn_3 = nn.Conv2d(in_channels=5,out_channels=5,kernel_size=1,
                                padding=0,bias=False)
        self.bn_3 = nn.BatchNorm2d(5)
        self.relu_3 = nn.ReLU()
        self.maxpool_3 = nn.MaxPool2d(kernel_size=2,stride=None) 
     
        self.cnn_d4 = nn.Conv2d(in_channels=5,out_channels=5,kernel_size=3,
                                padding=0,groups=5,bias=False)
        self.bn_d4 = nn.BatchNorm2d(5)
        self.relu_d4 = nn.ReLU()
        self.cnn_4 = nn.Conv2d(in_channels=5,out_channels=3,kernel_size=1,
                                padding=0,bias=False)
        self.bn_4 = nn.BatchNorm2d(3)
        self.relu_4 = nn.ReLU()
        self.maxpool_4 = nn.MaxPool2d(kernel_size=3,stride=None) 
        
        self.drop = nn.Dropout(p=0.5)
        self.dense =nn.Linear(2*2*3,2,bias=False)
        self.bn_5 = nn.BatchNorm1d(2)
        self.relu_5 = nn.ReLU()
        
    def forward(self,x):
        
        x = self.relu_1(self.bn_1(self.cnn_1(x)))
        x = self.maxpool_1(x)
            
        x = self.relu_d2(self.bn_d2(self.cnn_d2(x)))
        x = self.relu_2(self.bn_2(self.cnn_2(x)))

        x = self.relu_d3(self.bn_d3(self.cnn_d3(x)))
        x = self.relu_3(self.bn_3(self.cnn_3(x)))
        x = self.maxpool_3(x)
        
        x = self.relu_d4(self.bn_d4(self.cnn_d4(x)))
        x = self.relu_4(self.bn_4(self.cnn_4(x)))
        x = self.maxpool_4(x)
            
        x = x.view(x.size(0),-1)
        x = self.drop(x)
        x = self.dense(x)
        x = self.bn_5(x)
        x = self.relu_5(x)

        return x

In [99]:
ECG_AF_2D = ECG_AF_2D_Model()
summary(ECG_AF_2D,(1,73,73))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 36, 36]              54
       BatchNorm2d-2            [-1, 6, 36, 36]              12
              ReLU-3            [-1, 6, 36, 36]               0
         MaxPool2d-4            [-1, 6, 18, 18]               0
            Conv2d-5            [-1, 6, 16, 16]              54
       BatchNorm2d-6            [-1, 6, 16, 16]              12
              ReLU-7            [-1, 6, 16, 16]               0
            Conv2d-8            [-1, 5, 16, 16]              30
       BatchNorm2d-9            [-1, 5, 16, 16]              10
             ReLU-10            [-1, 5, 16, 16]               0
           Conv2d-11            [-1, 5, 16, 16]              45
      BatchNorm2d-12            [-1, 5, 16, 16]              10
             ReLU-13            [-1, 5, 16, 16]               0
           Conv2d-14            [-1, 5,

In [6]:
# without depthwise

class ECG_AF_2D_Model(nn.Module):
    def __init__(self):
        super(ECG_AF_2D_Model,self).__init__()
        
        self.cnn_1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=2,
                                padding=0,bias=False)
        self.bn_1 = nn.BatchNorm2d(6)
        self.relu_1 = nn.ReLU()
        self.maxpool_1 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn_d2 = nn.Conv2d(in_channels=6,out_channels=5,kernel_size=3,
                                padding=0,bias=False)
        self.bn_d2 = nn.BatchNorm2d(5)
        self.relu_d2 = nn.ReLU()
        
        self.cnn_d3 = nn.Conv2d(in_channels=5,out_channels=5,kernel_size=3,
                                padding=1,bias=False)
        self.bn_d3 = nn.BatchNorm2d(5)
        self.relu_d3 = nn.ReLU()
        self.maxpool_3 = nn.MaxPool2d(kernel_size=2,stride=None) 
     
        self.cnn_d4 = nn.Conv2d(in_channels=5,out_channels=3,kernel_size=3,
                                padding=0,bias=False)
        self.bn_d4 = nn.BatchNorm2d(3)
        self.relu_d4 = nn.ReLU()
        self.maxpool_4 = nn.MaxPool2d(kernel_size=3,stride=None) 
        
        self.drop = nn.Dropout(p=0.5)
        self.dense =nn.Linear(2*2*3,2,bias=False)
        self.bn_5 = nn.BatchNorm1d(2)
        self.relu_5 = nn.ReLU()
        
    def forward(self,x):
        
        x = self.relu_1(self.bn_1(self.cnn_1(x)))
        x = self.maxpool_1(x)
            
        x = self.relu_d2(self.bn_d2(self.cnn_d2(x)))

        x = self.relu_d3(self.bn_d3(self.cnn_d3(x)))
        x = self.maxpool_3(x)
        
        x = self.relu_d4(self.bn_d4(self.cnn_d4(x)))
        x = self.maxpool_4(x)
            
        x = x.view(x.size(0),-1)
        x = self.drop(x)
        x = self.dense(x)
        x = self.bn_5(x)
        x = self.relu_5(x)

        return x

In [7]:
ECG_AF_2D = ECG_AF_2D_Model()
summary(ECG_AF_2D,(1,73,73))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 36, 36]              54
       BatchNorm2d-2            [-1, 6, 36, 36]              12
              ReLU-3            [-1, 6, 36, 36]               0
         MaxPool2d-4            [-1, 6, 18, 18]               0
            Conv2d-5            [-1, 5, 16, 16]             270
       BatchNorm2d-6            [-1, 5, 16, 16]              10
              ReLU-7            [-1, 5, 16, 16]               0
            Conv2d-8            [-1, 5, 16, 16]             225
       BatchNorm2d-9            [-1, 5, 16, 16]              10
             ReLU-10            [-1, 5, 16, 16]               0
        MaxPool2d-11              [-1, 5, 8, 8]               0
           Conv2d-12              [-1, 3, 6, 6]             135
      BatchNorm2d-13              [-1, 3, 6, 6]               6
             ReLU-14              [-1, 

# MNIST_mAlexNet

## size (28,28)

### 2D

In [100]:
class mAlexNet_2D_28_Model(nn.Module):
    def __init__(self):
        super(mAlexNet_2D_28_Model,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,
                                padding=1,bias=False)
        self.bn_1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn2 = nn.Conv2d(in_channels=16,out_channels=20,kernel_size=3,
                                padding=0,bias=False)
        self.relu2 = nn.ReLU()
        self.bn_2 = nn.BatchNorm2d(20)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn3 = nn.Conv2d(in_channels=20,out_channels=30,kernel_size=3,
                                padding=0,bias=False)
        self.relu3 = nn.ReLU()
        self.bn_3 = nn.BatchNorm2d(30)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(2*2*30,60,bias=False)
        self.relu4 = nn.ReLU()
        
        self.fc2 = nn.Linear(60,10,bias=False)
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        x = self.maxpool3(self.relu3(self.cnn3(x)))
        
        x = x.view(x.size(0),-1)
        x = self.drop1(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [101]:
mAlexNet_2D_28 = mAlexNet_2D_28_Model()
summary(mAlexNet_2D_28,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 20, 12, 12]           2,880
              ReLU-5           [-1, 20, 12, 12]               0
         MaxPool2d-6             [-1, 20, 6, 6]               0
            Conv2d-7             [-1, 30, 4, 4]           5,400
              ReLU-8             [-1, 30, 4, 4]               0
         MaxPool2d-9             [-1, 30, 2, 2]               0
          Dropout-10                  [-1, 120]               0
           Linear-11                   [-1, 60]           7,200
             ReLU-12                   [-1, 60]               0
           Linear-13                   [-1, 10]             600
Total params: 16,224
Trainable params: 

### 1D

In [113]:
class mAlexNet_1D_784_Model(nn.Module):
    def __init__(self):
        super(mAlexNet_1D_784_Model,self).__init__()
        
        self.cnn1 = nn.Conv1d(in_channels=1,out_channels=16,kernel_size=9,
                                padding=0,bias=False)
        self.bn_1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=4,stride=None)
        
        self.cnn2 = nn.Conv1d(in_channels=16,out_channels=20,kernel_size=9,
                                padding=0,bias=False)
        self.relu2 = nn.ReLU()
        self.bn_2 = nn.BatchNorm1d(20)
        self.maxpool2 = nn.MaxPool1d(kernel_size=6,stride=None)
        
        self.cnn3 = nn.Conv1d(in_channels=20,out_channels=30,kernel_size=8,
                                padding=0,bias=False)
        self.relu3 = nn.ReLU()
        self.bn_3 = nn.BatchNorm1d(30)
        self.maxpool3 = nn.MaxPool1d(kernel_size=6,stride=None)
        
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(4*30,60,bias=False)
        self.relu4 = nn.ReLU()
        
        self.fc2 = nn.Linear(60,10,bias=False)
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        x = self.maxpool3(self.relu3(self.cnn3(x)))
        
        x = x.view(x.size(0),-1)
        x = self.drop1(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [114]:
mAlexNet_1D_784 = mAlexNet_1D_784_Model()
summary(mAlexNet_1D_784,(1,784))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 16, 776]             144
              ReLU-2              [-1, 16, 776]               0
         MaxPool1d-3              [-1, 16, 194]               0
            Conv1d-4              [-1, 20, 186]           2,880
              ReLU-5              [-1, 20, 186]               0
         MaxPool1d-6               [-1, 20, 31]               0
            Conv1d-7               [-1, 30, 24]           4,800
              ReLU-8               [-1, 30, 24]               0
         MaxPool1d-9                [-1, 30, 4]               0
          Dropout-10                  [-1, 120]               0
           Linear-11                   [-1, 60]           7,200
             ReLU-12                   [-1, 60]               0
           Linear-13                   [-1, 10]             600
Total params: 15,624
Trainable params: 

## (196,196)

In [4]:
class mAlexNet_2D_196_Model(nn.Module):
    def __init__(self):
        super(mAlexNet_2D_196_Model,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,
                                padding=1,bias=False)
        self.bn_1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn2 = nn.Conv2d(in_channels=16,out_channels=20,kernel_size=3,
                                padding=1,bias=False)
        self.relu2 = nn.ReLU()
        self.bn_2 = nn.BatchNorm2d(20)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn3 = nn.Conv2d(in_channels=20,out_channels=30,kernel_size=3,
                                padding=1,bias=False)
        self.relu3 = nn.ReLU()
        self.bn_3 = nn.BatchNorm2d(30)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(24*24*30,60,bias=False)
        self.relu4 = nn.ReLU()
        
        self.fc2 = nn.Linear(60,10,bias=False)
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        x = self.maxpool3(self.relu3(self.cnn3(x)))
        
        x = x.view(x.size(0),-1)
        x = self.drop1(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [5]:
mAlexNet_2D_196 = mAlexNet_2D_196_Model()
summary(mAlexNet_2D_196,(1,196,196))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 196, 196]             144
              ReLU-2         [-1, 16, 196, 196]               0
         MaxPool2d-3           [-1, 16, 98, 98]               0
            Conv2d-4           [-1, 20, 98, 98]           2,880
              ReLU-5           [-1, 20, 98, 98]               0
         MaxPool2d-6           [-1, 20, 49, 49]               0
            Conv2d-7           [-1, 30, 49, 49]           5,400
              ReLU-8           [-1, 30, 49, 49]               0
         MaxPool2d-9           [-1, 30, 24, 24]               0
          Dropout-10                [-1, 17280]               0
           Linear-11                   [-1, 60]       1,036,800
             ReLU-12                   [-1, 60]               0
           Linear-13                   [-1, 10]             600
Total params: 1,045,824
Trainable param

In [8]:
class mAlexNet_1D_38416_Model(nn.Module):
    def __init__(self):
        super(mAlexNet_1D_38416_Model,self).__init__()
        
        self.cnn1 = nn.Conv1d(in_channels=1,out_channels=16,kernel_size=8,
                                padding=0,bias=False)
        self.bn_1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=4,stride=None)
        
        self.cnn2 = nn.Conv1d(in_channels=16,out_channels=20,kernel_size=6,
                                padding=0,bias=False)
        self.relu2 = nn.ReLU()
        self.bn_2 = nn.BatchNorm1d(20)
        self.maxpool2 = nn.MaxPool1d(kernel_size=4,stride=None)
        
        self.cnn3 = nn.Conv1d(in_channels=20,out_channels=30,kernel_size=7,
                                padding=0,bias=False)
        self.relu3 = nn.ReLU()
        self.bn_3 = nn.BatchNorm1d(30)
        self.maxpool3 = nn.MaxPool1d(kernel_size=4,stride=None)
        
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(598*30,60,bias=False)
        self.relu4 = nn.ReLU()
        
        self.fc2 = nn.Linear(60,10,bias=False)
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        x = self.maxpool3(self.relu3(self.cnn3(x)))
        
        x = x.view(x.size(0),-1)
        x = self.drop1(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [9]:
mAlexNet_1D_38416 = mAlexNet_1D_38416_Model()
summary(mAlexNet_1D_38416,(1,38416))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1            [-1, 16, 38409]             128
              ReLU-2            [-1, 16, 38409]               0
         MaxPool1d-3             [-1, 16, 9602]               0
            Conv1d-4             [-1, 20, 9597]           1,920
              ReLU-5             [-1, 20, 9597]               0
         MaxPool1d-6             [-1, 20, 2399]               0
            Conv1d-7             [-1, 30, 2393]           4,200
              ReLU-8             [-1, 30, 2393]               0
         MaxPool1d-9              [-1, 30, 598]               0
          Dropout-10                [-1, 17940]               0
           Linear-11                   [-1, 60]       1,076,400
             ReLU-12                   [-1, 60]               0
           Linear-13                   [-1, 10]             600
Total params: 1,083,248
Trainable param

## with stride
## size (28,28)

In [116]:
class mAlexNet_2D_28_s_Model(nn.Module):
    def __init__(self):
        super(mAlexNet_2D_28_s_Model,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,
                                padding=1,bias=False)
        self.bn_1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn2 = nn.Conv2d(in_channels=16,out_channels=20,kernel_size=3,
                                padding=0,bias=False)
        self.relu2 = nn.ReLU()
        self.bn_2 = nn.BatchNorm2d(20)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        self.cnn3 = nn.Conv2d(in_channels=20,out_channels=30,kernel_size=3,stride=3,
                                padding=0,bias=False)
        self.relu3 = nn.ReLU()
        self.bn_3 = nn.BatchNorm2d(30)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride=None)
        
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(1*1*30,60,bias=False)
        self.relu4 = nn.ReLU()
        
        self.fc2 = nn.Linear(60,10,bias=False)
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        x = self.maxpool3(self.relu3(self.cnn3(x)))
        
        x = x.view(x.size(0),-1)
        x = self.drop1(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [2]:
class mAlexNet_1D_784_s_Model(nn.Module):
    def __init__(self):
        super(mAlexNet_1D_784_s_Model,self).__init__()
        
        self.cnn1 = nn.Conv1d(in_channels=1,out_channels=16,kernel_size=9,
                                padding=0,bias=False)
        self.bn_1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=4,stride=None)
        
        self.cnn2 = nn.Conv1d(in_channels=16,out_channels=20,kernel_size=9,
                                padding=0,bias=False)
        self.relu2 = nn.ReLU()
        self.bn_2 = nn.BatchNorm1d(20)
        self.maxpool2 = nn.MaxPool1d(kernel_size=6,stride=None)
        
        self.cnn3 = nn.Conv1d(in_channels=20,out_channels=30,kernel_size=7,stride=8,
                                padding=0,bias=False)
        self.relu3 = nn.ReLU()
        self.bn_3 = nn.BatchNorm1d(30)
        self.maxpool3 = nn.MaxPool1d(kernel_size=4,stride=None)
        
        
        self.drop1 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(1*30,60,bias=False)
        self.relu4 = nn.ReLU()
        
        self.fc2 = nn.Linear(60,10,bias=False)
        
       
    def forward(self,x):
        x = self.maxpool1(self.relu1(self.cnn1(x)))
        x = self.maxpool2(self.relu2(self.cnn2(x)))
        x = self.maxpool3(self.relu3(self.cnn3(x)))
        
        x = x.view(x.size(0),-1)
        x = self.drop1(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [118]:
mAlexNet_2D_28_s = mAlexNet_2D_28_s_Model()
summary(mAlexNet_2D_28_s,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 20, 12, 12]           2,880
              ReLU-5           [-1, 20, 12, 12]               0
         MaxPool2d-6             [-1, 20, 6, 6]               0
            Conv2d-7             [-1, 30, 2, 2]           5,400
              ReLU-8             [-1, 30, 2, 2]               0
         MaxPool2d-9             [-1, 30, 1, 1]               0
          Dropout-10                   [-1, 30]               0
           Linear-11                   [-1, 60]           1,800
             ReLU-12                   [-1, 60]               0
           Linear-13                   [-1, 10]             600
Total params: 10,824
Trainable params: 

In [3]:
mAlexNet_1D_784_s = mAlexNet_1D_784_s_Model()
summary(mAlexNet_1D_784_s,(1,784))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 16, 776]             144
              ReLU-2              [-1, 16, 776]               0
         MaxPool1d-3              [-1, 16, 194]               0
            Conv1d-4              [-1, 20, 186]           2,880
              ReLU-5              [-1, 20, 186]               0
         MaxPool1d-6               [-1, 20, 31]               0
            Conv1d-7                [-1, 30, 4]           4,200
              ReLU-8                [-1, 30, 4]               0
         MaxPool1d-9                [-1, 30, 1]               0
          Dropout-10                   [-1, 30]               0
           Linear-11                   [-1, 60]           1,800
             ReLU-12                   [-1, 60]               0
           Linear-13                   [-1, 10]             600
Total params: 9,624
Trainable params: 9

D:\Anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
